#### Topics:
    Views

A view is a named query stored in the database catalog that allows you to refer to it later. 

![Views](TSQL-Views.png)

In [1]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
#[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=BikeStores;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Create a cursor
cursor = conn.cursor()

In [3]:
cursor.execute('''
SELECT
    product_name, 
    brand_name, 
    list_price
FROM
    production.products p
INNER JOIN production.brands b 
        ON b.brand_id = p.brand_id;

''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,brand_name,list_price
0,Trek 820 - 2016,Trek,379.99
1,Ritchey Timberwolf Frameset - 2016,Ritchey,749.99
2,Surly Wednesday Frameset - 2016,Surly,999.99
3,Trek Fuel EX 8 29 - 2016,Trek,2899.99
4,Heller Shagamaw Frame - 2016,Heller,1320.99
5,Surly Ice Cream Truck Frameset - 2016,Surly,469.99
6,Trek Slash 8 27.5 - 2016,Trek,3999.99
7,Trek Remedy 29 Carbon Frameset - 2016,Trek,1799.99
8,Trek Conduit+ - 2016,Trek,2999.99
9,Surly Straggler - 2016,Surly,1549.00


Next time, if you want to get the same result set, you can save this query into a text file, open it, and execute it again.

SQL Server provides a better way to save this query in the database catalog through a view.

A view is a named query stored in the database catalog that allows you to refer to it later.

In [6]:
cursor.execute('''
CREATE VIEW sales.product_information
AS
SELECT
    product_name, 
    brand_name, 
    list_price
FROM
    production.products p
INNER JOIN production.brands b 
        ON b.brand_id = p.brand_id;

''')


In [7]:
cursor.execute('''
SELECT * FROM sales.product_information
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,brand_name,list_price
0,Trek 820 - 2016,Trek,379.99
1,Ritchey Timberwolf Frameset - 2016,Ritchey,749.99
2,Surly Wednesday Frameset - 2016,Surly,999.99
3,Trek Fuel EX 8 29 - 2016,Trek,2899.99
4,Heller Shagamaw Frame - 2016,Heller,1320.99
5,Surly Ice Cream Truck Frameset - 2016,Surly,469.99
6,Trek Slash 8 27.5 - 2016,Trek,3999.99
7,Trek Remedy 29 Carbon Frameset - 2016,Trek,1799.99
8,Trek Conduit+ - 2016,Trek,2999.99
9,Surly Straggler - 2016,Surly,1549.00


By definition, views do not store data except for indexed views.

A view may consist of columns from multiple tables using joins or just a subset of columns of a single table. This makes views useful for abstracting or hiding complex queries.

#### Advantages of views
Generally speaking, views provide the following advantages:

##### Security
You can restrict users to access directly to a table and allow them to access a subset of data via views.

For example, you can allow users to access customer name, phone, email via a view but restrict them to access the bank account and other sensitive information.

##### Simplicity
A relational database may have many tables with complex relationships e.g., one-to-one and one-to-many that make it difficult to navigate.

However, you can simplify the complex queries with joins and conditions using a set of views.

##### Consistency
Sometimes, you need to write a complex formula or logic in every query.

To make it consistent, you can hide the complex queries logic and calculations in views.

Once views are defined, you can reference the logic from the views rather than rewriting it in separate queries.

#### CREATING OR ALTERING VIEW

In [10]:
cursor.execute('''
CREATE VIEW sales.daily_sales_info
AS
SELECT
    year(order_date) AS y,
    month(order_date) AS m,
    day(order_date) AS d,
    p.product_id,
    product_name,
    quantity * i.list_price AS sales
FROM
    sales.orders AS o
INNER JOIN sales.order_items AS i
    ON o.order_id = i.order_id
INNER JOIN production.products AS p
    ON p.product_id = i.product_id;

''')


In [14]:
cursor.execute(''' 
SELECT 
    * 
FROM 
    sales.daily_sales_info
ORDER BY
    y, m, d, product_name;

'''
)


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,y,m,d,product_id,product_name,sales
0,2016,1,1,16,Electra Townie Original 7D EQ - 2016,1199.98
1,2016,1,1,16,Electra Townie Original 7D EQ - 2016,1199.98
2,2016,1,1,20,Electra Townie Original 7D EQ - Women's - 2016,599.99
3,2016,1,1,20,Electra Townie Original 7D EQ - Women's - 2016,599.99
4,2016,1,1,10,Surly Straggler - 2016,3098.00
5,2016,1,1,4,Trek Fuel EX 8 29 - 2016,2899.99
6,2016,1,1,8,Trek Remedy 29 Carbon Frameset - 2016,3599.98
7,2016,1,2,20,Electra Townie Original 7D EQ - Women's - 2016,599.99
8,2016,1,2,3,Surly Wednesday Frameset - 2016,999.99
9,2016,1,3,26,Electra Townie Original 7D EQ - 2016,599.99


In [22]:
cursor.execute('''
CREATE OR ALTER VIEW sales.daily_sales_info
AS
SELECT
    YEAR(order_date) AS year,
    MONTH(order_date) AS month,
    DAY(order_date) AS day,
    CONCAT(first_name, ' ', last_name) AS customer_name,
    p.product_id,
    p.product_name,
    quantity * i.list_price AS sales
FROM
    sales.orders AS o
    INNER JOIN sales.order_items AS i ON o.order_id = i.order_id
    INNER JOIN production.products AS p ON p.product_id = i.product_id
    INNER JOIN sales.customers AS c ON c.customer_id = o.customer_id;

''')


In [25]:
cursor.execute(''' 
SELECT 
    * 
FROM 
    sales.daily_sales_info
ORDER BY
    year, month, day, product_name;

'''
)


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,year,month,day,customer_name,product_id,product_name,sales
0,2016,1,1,Johnathan Velazquez,16,Electra Townie Original 7D EQ - 2016,1199.98
1,2016,1,1,Jaqueline Cummings,16,Electra Townie Original 7D EQ - 2016,1199.98
2,2016,1,1,Jaqueline Cummings,20,Electra Townie Original 7D EQ - Women's - 2016,599.99
3,2016,1,1,Johnathan Velazquez,20,Electra Townie Original 7D EQ - Women's - 2016,599.99
4,2016,1,1,Johnathan Velazquez,10,Surly Straggler - 2016,3098.00
5,2016,1,1,Johnathan Velazquez,4,Trek Fuel EX 8 29 - 2016,2899.99
6,2016,1,1,Johnathan Velazquez,8,Trek Remedy 29 Carbon Frameset - 2016,3599.98
7,2016,1,2,Joshua Robertson,20,Electra Townie Original 7D EQ - Women's - 2016,599.99
8,2016,1,2,Joshua Robertson,3,Surly Wednesday Frameset - 2016,999.99
9,2016,1,3,Arla Ellis,26,Electra Townie Original 7D EQ - 2016,599.99
